In [1]:
import arviz
import cmdstanpy
from IPython.display import display
from matplotlib import pyplot as plt
from numbers import Number
import numpy as np
import os
import pandas as pd
from pandas.io.json import json_normalize
import toml
import reaction_network
plt.style.use('sparse.mplstyle')

# Modelling networks of metabolic reactions with Stan

Many useful substances can be produced using chemical reactions that happen inside living organisms like _E. Coli,_ yeast or Chinese hamster ovaries. Biological production is appealing because it can be done in relatively mild conditions. On the other hand, biological production depends on complex networks of interacting enzyme-catalysed chemical reactions which can be difficult to understand, manipulate and optimise.

This notebook describes how to analyse metabolic networks by integrating different sources of information - structural knowledge, pre-experimental information about parameters and experiment results - using Bayesian inference.

## Wait, what is an enzyme-catalysed reaction?
An enzyme is a 'catalyst' protein that speeds up a chemical reaction by binding the reactants to an 'active site' where the conditions for the reaction are favourable. This visualisation gives an idea of how this process is thought to work: 

![Enzyme Action and the Hydrolysis of Sucrose [HD Animation]](https://i.makeagif.com/media/9-16-2015/SIif27.gif)

## Structural information 

The topological structure of most industrially relevant metabolic networks has been mapped. It is usually known which substances go in and out of each reaction the enzyme that catalyses it. Often there is even more structural information, such as what shape the enzyme has, which substances can bind to its active site.

This information is available online, for example in the [BIGG database](http://bigg.ucsd.edu/). 

Here is a reduced map of yeast central metabolism, with nodes representing metabolites and edges representing reactions: 

<img src="img/yeast_central_metabolism.png" alt="Drawing" style="width: 600px;"/>

Finer-grained structural information can be used to deduce the mechanism by which each enzyme operates. A mechanism, in turn, implies a way of parameterising the rate at which an enzyme turns substrates into products. For example, the following equation describes how the rate $v$ of a 'uni-uni' mechanism (See [Cleland (1963)](https://doi.org/10.1016/0926-6569%2863%2990211-6) for more about mechanisms in general and derivations of particular mechanisms used in this analysis) with no activators or inhibitors depends on the current concentration of its substrate $A$ and product $P$, as well as parameters $K_{eq}$, $K_{a}$, $V1$ and $V2$:

$$
\begin{equation}
v = \frac{V1\cdot{V2}\cdot{(A-P/K_{eq})}}{K_{a}\cdot{V2} + V2\cdot{A} + V1\cdot{P/K_{eq}}}
\end{equation}
$$

The main output of this analysis will be inference about the values of this kind of parameter.


## Pre-experimental information
Most parameters are difficult to assess because the conditions inside a cell are difficult to replicate _in vitro_. See [Saa and Keld Nielsen (2017)](https://doi.org/10.1016/j.biotechadv.2017.09.005) for more about top-down vs bottom-up parameter estimation. Nonetheless, a lot of information is available.

For kinetic parameters it is possible to use the 'Respectful' modelling paradigm (see [Tsigkinopoulou, Baker, and Breitling (2017)](https://doi.org/10.1016/j.tibtech.2016.12.008)) to aggregate the results of findings about kinetic parameters published in the [BRENDA enzyme database](https://www.brenda-enzymes.org/), as well as assessments of thermodynamic parameters from the [equilibrator database](http://equilibrator.weizmann.ac.il/). This analysis produces an independent normal or log-normal distribution representing the pre-experimental information available about each parameter.

## Experimental information

Modern experimental techniques measure the following quantities:

- Enzyme concentrations (_proteomics_)
- Metabolite concentrations (_metabolomics_)
- Rates of reactions, aka fluxes (_fluxomics_)

Each kind of measurement is noisy.

## Modelling strategy

We are proposing to use Bayesian inference to combine all the available sources of information about a metabolic network.

Pre-experimental information supplies informative prior probability distributions about the parameters that describe each reaction.

We treat information from metabolomics and fluxomics data using a regression model where recorded values of measured quantities are draws from distributions centered on the true values as follows, with indexes $e$ standing for experiments, $f$ for fluxes and $m$ for metabolites:

$$
measured\, flux_{f,e} \sim Normal(true\, flux_{f,e}, \sigma_{f,e}) \\
measured\, metabolite\, concentration_{m,e} \sim Lognormal(\log(true\, metabolite\, concentration_{m,e}), \sigma_{m,e})
$$

Structural information can be used to connect the other two sources of information by mapping parameter configurations to true metabolite concentrations and fluxes. This is done by expressing the evolution of each metabolite's concentration as a system of ordinary differential equations, with the rate of change of each metabolite determined by the rates of the reactions that produce and consume it. We assume that, at the time of measurement, the concentration of the network's internal metabolites is constant, or in other words that the system has reached a steady state. In general, a vector $\mathbf{v}$ of steady state fluxes must satisfy the following condition

$$
S\cdot\mathbf{v} = \mathbf{0}
$$

Where $S$ is a known 'stoichiometric matrix' indicating which reactions produce and consume which metabolites and $\mathbf{0}$ is a vector of as many zeros as the network has internal metabolites. Given a parameter configuration, a steady state metabolite concentration can be found by solving the resulting system of algebraic equations. 

We solve these equation systems by integrating the implied ODEs over a period of time for which it is safe to assume that the system has reached steady state.

This approach is essentially the same as that set out by [Saa and Keld Nielsen (2016)](https://doi.org/10.1038/srep29635), but uses MCMC via Stan rather than rejection sampling to make inferences. Thanks to Stan's intuitive model language, this approach and attendant assumptions can be represented in a more or less self-documenting form.


## Example - a simple reaction network

The rest of the notebook applies this modelling strategy to investigate how accurately it is possible to find out the value of the kinetic parameters and steady state metabolite concentrations of the simple metabolic network depicted below, given a known experimental setup and prior information.

<img src="img/toy_model_pic.png" alt="Drawing" style="width: 800px;"/>

This network has four internal metabolites, M1-4, and six reactions R1-6. Metabolites M2, M3 and M4 are measured, and reaction R1 is inhibited by M4. The metabolites with 'ex' in their names are external, and are modelled differently as they are not assumed to have constant concentrations at steady state.

To gauge how accurately the available information identifies the quantities of interest, the notebook uses Stan to generate simulated experimental data given 'true' parameter values, then uses the simulated data as model input. If the model is unable to recover the true value of a parameter, this is may be a sign that the simulated experiment shouldn't be carried out in real life without more diverse or accurate measurements or better pre-experimental information.

## Model input
We are developing an input data format for succinctly expressing the information required to model a metabolic network. Here is what the linear system looks like in this format:

In [2]:
with open('toy.toml', 'r') as f:
    print(f.read())

[constants]
gas_constant = 8.314e-3 # units kJ/mol.K

[[experiment]]
label = 'condition_1'
concentration_r1 = 1
concentration_r2 = 1.5
concentration_r3 = 2
concentration_r4 = 1.5
concentration_r5 = 1.5
concentration_r6 = 1
unbalanced_metabolite_priors = [
  { label = 'M1_ex', loc = 1, scale = 0.4, true_value = 6 },
  { label = 'M2_ex', loc = 2, scale = 0.4, true_value = 1 },
  { label = 'M4_ex', loc = 3, scale = 0.4, true_value = 1 }
]
measurements = [
  { label = 'M1', value = 2.28, scale = 0.1, type = 'metabolite' },
  { label = 'M2', value = 0.29, scale = 0.05, type = 'metabolite' },
  { label = 'M3', value = 0.34, scale = 0.05, type = 'metabolite' },
  { label = 'M4', value = 0.30, scale = 0.05, type = 'metabolite' },
  { label = 'r1', value = 0.31, scale = 0.05, type = 'flux' }
]

[[experiment]]
label = 'condition_2'
concentration_r1 = 1
concentration_r2 = 1
concentration_r3 = 1
concentration_r4 = 1
concentration_r5 = 1
concentration_r6 = 1
unbalanced_metabolite_priors = [
  { lab

The `reaction` entries in this file jointly specify the network's stoichiometry, reaction mechanisms, priors and allosteric interactions, while the `experiment` entries describe how the system has been measured, including experimental conditions, measured quantities and accuracies of each measurement. A value for each measurement can also be specified, but here we will simulate these using the specified parameter `true_value`s. It is assumed that all experiments were carried out at the same temperature.

### Allosteric regulation

Allosteric effects occur when an enzyme molecule's shape changes due to substances binding to it, making the enzyme a more or less effective catalyst. Allosteric effects play an important role in regulating metabolic reactions. Following [Popova and Selkov (1975)](https://doi.org/10.1016/0014-5793(75)80034-2), we model allosteric regulation through a function that expresses the overall effect of allosteric factors on the rate of a reaction as a multiplicative factor. Here is what the function looks like in Stan code:

In [3]:
with open('stan_code/allostery.stan', 'r') as f:
    print(f.read())

real get_free_enzyme_ratio_uniuni(real A, real P, real V1, real V2, real Ka, real Keq){
  real Kp = Keq * V2 * Ka / V1;
  return 1 / (1 + A/Ka + P/Kp);
}
real get_regulatory_effect(real[] activator_concentration,
                           real[] inhibitor_concentration,
                           real free_enzyme_ratio,
                           real[] dissociation_constant_r,
                           real[] dissociation_constant_t,
                           real transfer_constant){
  real Q_num = size(inhibitor_concentration) == 0 ? 1 :
    1 + sum(to_vector(inhibitor_concentration) ./ to_vector(dissociation_constant_t));
  real Q_denom = size(activator_concentration) == 0 ? 1
    : 1 + sum(to_vector(activator_concentration) ./ to_vector(dissociation_constant_r));
  real Q = transfer_constant * free_enzyme_ratio * Q_num / Q_denom;
  return inv(1 + Q); 
}
    



### Rate equations

The following Stan functions describe the catalytic effects of each reaction mechanism that appears in the network we want to analyse

In [5]:
with open('stan_code/rate_equations.stan', 'r') as f:
    print(f.read())

real uniuni(real A, real P, real V1, real V2, real Ka, real Keq){
  real num = V1*V2*(A-P/Keq);
  real denom = Ka*V2 + V2*A + V1*P/Keq;
  return num / denom;
}
real ordered_unibi(real A, real P, real Q,
                   real V1, real V2,
                   real Ka, real Kp, real Kq,
                   real Kia, real Kip, real Kiq,
                   real Keq){
  real num = V1*V2*(A-P*Q/Keq); 
  real denom =
    Ka*V2
    + V2*A
    + Kq*V1*P/Keq
    + Kp*V1*Q/Keq
    + V1*P*Q/Keq
    + V2*A*P/Kip;
  return num / denom;
}
real irr_mass_action(real A, real V1){
  return(A*V1);
}
real fixed_flux(real f){
  return f;
}



### Inference

The model for making inferences is shown below. The `functions` block uses the library functions described above to define an ode function called `steady_state_equation`. This function is called in the `transformed parameters` block in order to obtain metabolite concentrations given parameter values. In the `model` block the values of the variable `metabolite_concentration` are used in the measurement model.




In [6]:
with open('stan_code/inference_model_toy.stan', 'r') as f:
    print(f.read())

functions {
#include rate_equations.stan
#include haldane_relationships.stan
#include allostery.stan
real [] get_fluxes(real[] metabolites, real[] params, real[] known_reals){  
    real r2_Kip = get_Kip_ordered_unibi(params[5], params[11], params[10], params[6], params[7]);
    real r2_Kiq = get_Kiq_ordered_unibi(params[5], params[8], params[9], params[6], params[7]);

  

  return{   
      uniuni(metabolites[2], metabolites[1], known_reals[1]*params[2], known_reals[1]*params[3], params[4], params[1]),
      ordered_unibi(metabolites[1], metabolites[3], metabolites[4], known_reals[2]*params[6], known_reals[2]*params[7], params[8], params[9], params[10], params[11], r2_Kip, r2_Kiq, params[5]),
      uniuni(metabolites[4], metabolites[5], known_reals[3]*params[13], known_reals[3]*params[14], params[15], params[12]),
      uniuni(metabolites[5], metabolites[6], known_reals[4]*params[17], known_reals[4]*params[18], params[19], params[16]),
      uniuni(metabolites[3], metabolites[7], kno

### Simulating some  measurements
A separate simulation model, shown below, is used to generate measurement values. It has the same `functions` block as the inference model, but does not do any MCMC as no uncertainty needs to be taken into account.

In [8]:
with open('stan_code/simulation_model_toy.stan', 'r') as f:
    print(f.read())

functions {
#include rate_equations.stan
#include haldane_relationships.stan
#include allostery.stan
real [] get_fluxes(real[] metabolites, real[] params, real[] known_reals){  
    real r2_Kip = get_Kip_ordered_unibi(params[5], params[11], params[10], params[6], params[7]);
    real r2_Kiq = get_Kiq_ordered_unibi(params[5], params[8], params[9], params[6], params[7]);

  

  return{   
      uniuni(metabolites[2], metabolites[1], known_reals[1]*params[2], known_reals[1]*params[3], params[4], params[1]),
      ordered_unibi(metabolites[1], metabolites[3], metabolites[4], known_reals[2]*params[6], known_reals[2]*params[7], params[8], params[9], params[10], params[11], r2_Kip, r2_Kiq, params[5]),
      uniuni(metabolites[4], metabolites[5], known_reals[3]*params[13], known_reals[3]*params[14], params[15], params[12]),
      uniuni(metabolites[5], metabolites[6], known_reals[4]*params[17], known_reals[4]*params[18], params[19], params[16]),
      uniuni(metabolites[3], metabolites[7], kno

The next cell uses the `true_value` entries from the input data to simulate some measurement values.

In [140]:
STEADY_STATE_TIME = 200
ABS_TOL = 1e-12
REL_TOL = 1e-13
MAX_STEPS = int(1e9)
SIMULATION_CSV_PATH = 'output_data/simulation_output_toy.csv'
SIMULATION_MODEL_FILE = 'stan_code/simulation_model_toy.stan'


rn = reaction_network.load_from_toml('toy.toml')

print('\nUnbalanced metabolite priors:')
display(rn.unbalanced_metabolite_priors)
print('\nkinetic parameter input')
display(rn.kinetic_parameters)

metabolites = rn.metabolites
metabolite_names = rn.stoichiometry.columns
unbalanced_metabolites = rn.metabolites.query('is_unbalanced')
balanced_metabolites = rn.metabolites.query('~is_unbalanced')
reaction_names = rn.stoichiometry.index
concentration_unbalanced = (
    rn.unbalanced_metabolite_priors
    .set_index(['metabolite_code', 'experiment_code'])
    ['true_value']
    .unstack()
    .values
    .tolist()
)
simulation_input_data = {
    'N_balanced': len(balanced_metabolites),
    'N_unbalanced': len(unbalanced_metabolites),
    'N_kinetic_parameter': len(rn.kinetic_parameters),
    'N_reaction': len(reaction_names),
    'N_experiment': len(rn.experiments),
    'N_known_real': len(rn.known_reals),
    'N_flux_measurement': len(rn.flux_measurements),
    'N_concentration_measurement': len(rn.concentration_measurements),
    'pos_balanced': balanced_metabolites['stan_code'].values,
    'pos_unbalanced': unbalanced_metabolites['stan_code'].values,
    'ix_experiment_concentration_measurement': rn.concentration_measurements['experiment_code'].values,
    'ix_experiment_flux_measurement': rn.flux_measurements['experiment_code'].values,
    'ix_metabolite_concentration_measurement': rn.concentration_measurements['metabolite_code'].values,
    'ix_reaction_flux_measurement': rn.flux_measurements['reaction_code'].values,
    'flux_measurement_scale': rn.flux_measurements['scale'].values,
    'concentration_measurement_scale': rn.concentration_measurements['scale'].values,
    'kinetic_parameter': rn.kinetic_parameters['true_value'].values,
    'concentration_unbalanced': concentration_unbalanced,
    'known_reals': rn.known_reals.drop('stan_code', axis=1).values.tolist(),
    'initial_time': 0,
    'steady_time': STEADY_STATE_TIME,
    'rel_tol': REL_TOL,
    'abs_tol': ABS_TOL,
    'max_steps': MAX_STEPS
}

model = cmdstanpy.Model(stan_file=SIMULATION_MODEL_FILE)
model.compile(include_paths=['stan_code'])
    
stanfit = model.sample(
    data=simulation_input_data,
    csv_basename=SIMULATION_CSV_PATH,
    adapt_engaged=False,
    warmup_iters=0,
    sampling_iters=1,
    chains=1,
    cores=1
)

simulated_concentration_measurements = (
    rn.concentration_measurements[['metabolite_label', 'experiment_label']]
    .rename(columns={'metabolite_label': 'metabolite', 'experiment_label': 'experiment'})
    .assign(
        simulated_measurement=stanfit.get_drawset(['simulated_concentration_measurement']).iloc[0].values
    )
    .round(2)
)
simulated_flux_measurements = (
    rn.flux_measurements[['flux_label', 'experiment_label']]
    .rename(columns={'flux_label': 'reaction', 'experiment_label': 'experiment'})
    .assign(
        simulated_measurement=stanfit.get_drawset(['simulated_flux_measurement']).iloc[0].values
    )
    .round(2)

)

print('\nSimulation output:')
display(simulated_concentration_measurements)
display(simulated_flux_measurements)


Unbalanced metabolite priors:


,metabolite,loc,scale,true_value,experiment_label,metabolite_code,experiment_code
0,M1_ex,1,0.4,3.27,condition_1,2,1
1,M2_ex,0,0.4,0.89,condition_1,7,1
2,M4_ex,0,0.4,1.49,condition_1,6,1
3,M1_ex,1,0.4,2.43,condition_2,2,2
4,M2_ex,0,0.4,1.13,condition_2,7,2
5,M4_ex,0,0.4,0.56,condition_2,6,2



kinetic parameter input


,parameter,loc,scale,type,true_value,metabolite,reaction,is_allosteric,label,stan_code
0,Keq,0.0,0.2,thermodynamic,0.93,NaN,r1,0,r1_Keq,1
1,Kcat1,1.1,0.3,kinetic,2.61,NaN,r1,0,r1_Kcat1,2
2,Kcat2,-1.0,1.0,kinetic,1.03,NaN,r1,0,r1_Kcat2,3
3,Ka,0.0,0.5,kinetic,0.80,NaN,r1,0,r1_Ka,4
4,dissociation_constant_t,0.0,0.6,allosteric,1.11,M4,r1,1,r1_dissociation_constant_t_M4,5
5,transfer_constant,0.0,0.6,allosteric,0.71,NaN,r1,1,r1_transfer_constant,6
6,Keq,2.0,0.5,thermodynamic,2.19,NaN,r2,0,r2_Keq,7
7,Kcat1,0.5,0.5,kinetic,4.72,NaN,r2,0,r2_Kcat1,8
8,Kcat2,0.2,0.3,kinetic,0.64,NaN,r2,0,r2_Kcat2,9
9,Ka,0.6,0.4,kinetic,2.51,NaN,r2,0,r2_Ka,10


INFO:cmdstanpy:compiling c++
INFO:cmdstanpy:compiled model file: /Users/tedgro/Code/stancon_2019_metabolic_network_notebook/stan_code/simulation_model_toy
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1



Simulation output:


,metabolite,experiment,simulated_measurement
0,M1,condition_1,2.66
1,M2,condition_1,1.34
2,M3,condition_1,2.08
3,M4,condition_1,5.98
5,M1,condition_2,2.12
6,M2,condition_2,1.41
7,M3,condition_2,1.05
8,M4,condition_2,2.70


,reaction,experiment,simulated_measurement
4,r1,condition_1,0.08
9,r1,condition_2,0.07


### Sampling given simulated measurements

The next cell uses cmdstanpy to sample from the model, once ignoring the data for prior predictive checks and once using values simulated above.


In [98]:
INFERENCE_MODEL_FILE = 'stan_code/inference_model_toy.stan'

# Initialise and compile model
inference_model = cmdstanpy.Model(stan_file=INFERENCE_MODEL_FILE)
inference_model.compile(include_paths=['stan_code'])

# Define inputs for prior and posterior and save them as json files 
unbalanced_loc, unbalanced_scale = (
    rn.unbalanced_metabolite_priors
    .set_index(['metabolite_code', 'experiment_code'])
    [col]
    .unstack()
    .values
    .tolist()
    for col in ['loc', 'scale']
)

inference_input_prior = {
    'N_balanced': len(balanced_metabolites),
    'N_unbalanced': len(unbalanced_metabolites),
    'N_kinetic_parameter': len(rn.kinetic_parameters),
    'N_reaction': len(reaction_names),
    'N_experiment': len(rn.experiments),
    'N_known_real': len(rn.known_reals),
    'N_flux_measurement': len(rn.flux_measurements),
    'N_concentration_measurement': len(rn.concentration_measurements),
    'pos_balanced': balanced_metabolites['stan_code'].values,
    'pos_unbalanced': unbalanced_metabolites['stan_code'].values,
    'ix_experiment_concentration_measurement': rn.concentration_measurements['experiment_code'].values,
    'ix_experiment_flux_measurement': rn.flux_measurements['experiment_code'].values,
    'ix_metabolite_concentration_measurement': rn.concentration_measurements['metabolite_code'].values,
    'ix_reaction_flux_measurement': rn.flux_measurements['reaction_code'].values,
    
    # Simulated measurements used here
    'flux_measurement': simulated_flux_measurements['simulated_measurement'].values,
    'concentration_measurement': simulated_concentration_measurements['simulated_measurement'].values,
    
    'flux_measurement_scale': rn.flux_measurements['scale'].values,
    'concentration_measurement_scale': rn.concentration_measurements['scale'].values,
    'prior_location_kinetic_parameter': rn.kinetic_parameters['loc'].values,
    'prior_scale_kinetic_parameter': rn.kinetic_parameters['scale'].values,
    'prior_location_unbalanced': unbalanced_loc,
    'prior_scale_unbalanced': unbalanced_scale,
    'known_reals': rn.known_reals.drop('stan_code', axis=1).values.tolist(),
    'initial_time': 0,
    'steady_time': STEADY_STATE_TIME,
    'rel_tol': REL_TOL,
    'abs_tol': ABS_TOL,
    'max_steps': MAX_STEPS,
    'LIKELIHOOD': 0
}
inference_input_posterior = inference_input_prior.copy()
inference_input_posterior['LIKELIHOOD'] = 1

here = os.path.abspath('')
inference_input_file_prior = os.path.join(here, 'output_data/inference_input_data_prior.json')
inference_input_file_posterior = os.path.join(here, 'output_data/inference_input_data_posterior.json')
cmdstanpy.utils.jsondump(inference_input_file_prior, inference_input_prior)
cmdstanpy.utils.jsondump(inference_input_file_posterior, inference_input_posterior)

# Generate samples
inference_csv_basename_prior = os.path.join(here, 'output_data/inference_output_toy_prior.csv')
inference_csv_basename_posterior = os.path.join(here, 'output_data/inference_output_toy_posterior.csv')

stanfit_prior = inference_model.sample(
    data=inference_input_file_prior,
    csv_basename=inference_csv_basename_prior,
    warmup_iters=200,
    sampling_iters=100,
    chains=2,
    cores=2,
    save_warmup=True
)
stanfit_posterior = inference_model.sample(
    data=inference_input_file_posterior,
    csv_basename=inference_csv_basename_posterior,
    warmup_iters=200,
    sampling_iters=100,
    chains=2,
    cores=2,
    save_warmup=True
)

# Display summaries
display(stanfit_prior.summary())
display(stanfit_posterior.summary())
    
shared_arviz_kwargs = dict(
    posterior_predictive=[
        'simulated_flux_measurement', 
        'simulated_concentration_measurement'
    ],
    observed_data={
        'simulated_flux_measurement': inference_input_posterior['flux_measurement'],
        'simulated_concentration_measurement': inference_input_posterior['concentration_measurement']
    },
    coords={
        'reactions': reaction_names,
        'metabolites': metabolite_names,
        'experiments': [x['label'] for x in rn.experiments],
        'parameter_names':  (
        [f'{x}.{y}' for x, y in zip(rn.kinetic_parameters['reaction'], rn.kinetic_parameters['parameter'])]
        ),
        'flux_measurements': (
            [f'{x}.{y}' for x, y in zip(rn.flux_measurements['flux_label'], rn.flux_measurements['experiment_label'])]
        ),
        'concentration_measurements': (
            [f'{x}.{y}' for x, y in zip(rn.concentration_measurements['metabolite_label'], rn.concentration_measurements['experiment_label'])]
        )
    },
    dims={
        'concentration': ['metabolites', 'experiments'],
        'flux': ['reactions', 'experiments'],
        'kinetic_parameter': ['parameter_names'],
        'simulated_concentration_measurement': ['concentration_measurements'],
        'simulated_flux_measurement': ['flux_measurements'],

    }
)
infd_prior = arviz.from_cmdstanpy(stanfit_prior, **shared_arviz_kwargs)
infd_posterior = arviz.from_cmdstanpy(stanfit_posterior, **shared_arviz_kwargs)

INFO:cmdstanpy:compiling c++
INFO:cmdstanpy:compiled model file: /Users/tedgro/Code/stancon_2019_metabolic_network_notebook/stan_code/inference_model_toy
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 1


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-16.889100,0.350674,3.986290,-2.360850e+01,-1.634430e+01,-10.918800,129.220,0.863033,1.034170
kinetic_parameter[1],1.015350,0.010473,0.207702,7.013000e-01,9.905220e-01,1.382270,393.330,2.626960,0.995056
kinetic_parameter[2],3.189580,0.048231,1.034670,1.749130e+00,3.102500e+00,5.358660,460.206,3.073620,0.994372
kinetic_parameter[3],0.681654,0.109427,1.214200,8.323200e-02,3.635290e-01,1.950910,123.121,0.822297,1.011450
kinetic_parameter[4],1.107090,0.042067,0.628429,4.013030e-01,9.838910e-01,2.275130,223.165,1.490470,0.999826
kinetic_parameter[5],1.153600,0.033969,0.697676,3.588010e-01,1.005910e+00,2.551270,421.842,2.817390,0.996981
kinetic_parameter[6],1.207800,0.056439,0.876171,3.669490e-01,9.826080e-01,2.416670,240.998,1.609570,1.007400
kinetic_parameter[7],8.331270,0.369623,4.784800,2.904060e+00,6.839700e+00,18.935900,167.575,1.119200,0.996678
kinetic_parameter[8],1.802370,0.052246,0.953629,6.265630e-01,1.623440e+00,3.719890,333.155,2.225070,0.995647


NameError: name 'stanfit_posterior' is not defined

In [104]:
shared_arviz_kwargs = dict(
    posterior_predictive=[
        'simulated_flux_measurement', 
        'simulated_concentration_measurement'
    ],
    observed_data={
        'simulated_flux_measurement': inference_input_posterior['flux_measurement'],
        'simulated_concentration_measurement': inference_input_posterior['concentration_measurement']
    },
    coords={
        'reactions': reaction_names,
        'metabolites': metabolite_names,
        'experiments': [x['label'] for x in rn.experiments],
        'parameter_names':  (
        [f'{x}.{y}' for x, y in zip(rn.kinetic_parameters['reaction'], rn.kinetic_parameters['parameter'])]
        ),
        'flux_measurements': (
            [f'{x}.{y}' for x, y in zip(rn.flux_measurements['flux_label'], rn.flux_measurements['experiment_label'])]
        ),
        'concentration_measurements': (
            [f'{x}.{y}' for x, y in zip(rn.concentration_measurements['metabolite_label'], rn.concentration_measurements['experiment_label'])]
        )
    },
    dims={
        'concentration': ['metabolites', 'experiments'],
        'flux': ['reactions', 'experiments'],
        'kinetic_parameter': ['parameter_names'],
        'simulated_concentration_measurement': ['concentration_measurements'],
        'simulated_flux_measurement': ['flux_measurements'],

    }
)
infd_prior = arviz.from_cmdstanpy(stanfit_prior, **shared_arviz_kwargs)



In [133]:
infd_prior.posterior_predictive['simulated_flux_measurement'].to_series().sort_values(ascending=False).head(40).sort_index()

chain  draw  flux_measurements
0      7     r1.condition_1       0.146977
       11    r1.condition_1       0.093501
             r1.condition_2       0.102965
       14    r1.condition_1       0.115574
       16    r1.condition_1       0.118842
       18    r1.condition_2       0.132051
       20    r1.condition_2       0.091939
       21    r1.condition_2       0.159148
       26    r1.condition_1       0.095673
       43    r1.condition_1       0.134761
       46    r1.condition_1       0.172220
       47    r1.condition_1       0.093707
             r1.condition_2       0.089508
       48    r1.condition_1       0.099120
       52    r1.condition_1       0.111607
             r1.condition_2       0.118016
       56    r1.condition_2       0.121021
       61    r1.condition_1       0.098654
       70    r1.condition_1       0.117980
       71    r1.condition_1       0.099795
       72    r1.condition_2       0.094615
       77    r1.condition_1       0.169541
             r1.conditi

In [134]:
infd_prior.posterior_predictive.sel(chain=0, draw=88)['simulated_concentration_measurement'].to_series()

concentration_measurements
M1.condition_1    2.25855
M2.condition_1    1.56076
M3.condition_1    1.07587
M4.condition_1    3.41568
M1.condition_2    3.05341
M2.condition_2    1.23160
M3.condition_2    1.83038
M4.condition_2    5.42439
Name: simulated_concentration_measurement, dtype: float64

In [136]:
infd_prior.posterior.sel(chain=0, draw=88)['kinetic_parameter'].to_series()

parameter_names
r1.Keq                        0.939376
r1.Kcat1                      2.613380
r1.Kcat2                      1.037400
r1.Ka                         0.808946
r1.dissociation_constant_t    1.117330
r1.transfer_constant          0.715912
r2.Keq                        2.190050
r2.Kcat1                      4.722440
r2.Kcat2                      0.642321
r2.Ka                         2.514590
r2.Kp                         1.837770
r2.Kq                         3.746290
r2.Kia                        1.197060
r3.Keq                        1.606340
r3.Kcat1                      2.598260
r3.Kcat2                      0.263231
r3.Ka                         0.893458
r4.Keq                        0.963561
r4.Kcat1                      0.389483
r4.Kcat2                      0.836001
r4.Ka                         0.619424
r5.Keq                        0.943598
r5.Kcat1                      3.922340
r5.Kcat2                      1.856590
r5.Ka                         0.227546
r6.Keq   

### Model checking
From the summary and lack of warnings from the diagnostic function we can see that the chains have mixed pretty well and that there were no post-warmup divergent transitions. From the summary of the `metabolite_flux` variables shows that these were mostly very close to zero, indicating that the system reached steady state in most iterations.


### Analysis
In order to ascertain whether the simulated experimental apparatus is powerful enough for it to learn about the network's parameters, we can look at each parameter's marginal prior and posterior distribution. If the model has learned about the parameter then there should be more probability mass around the true value in the marginal posterior distribution than in the prior.

This cell uses the analysis package [arviz](https://arviz-devs.github.io/arviz/index.html) to plot some quantiles of each parameter's marginal prior and posterior distribution alongside the true value that was fed into the simulation.

For some parameters we can see that the marginal posterior concentrates more closely around the true value than the marginal prior distribution. For other parameters this is not the case. Depending on which parameters we care most about, this test could help determine whether or not to carry out the experiments.

In [ ]:
f, [ax] = arviz.plot_forest(
    [infd_prior, infd_posterior],
    combined=True,
    model_names=['prior', 'posterior'],
    var_names=['kinetic_parameter'],
    figsize=[15, 20],
    colors=['tab:orange', 'tab:blue']
)
t = ax.set_title('How did the marginal parameter distributions change from prior to posterior?', fontsize=15)
lines = [l for axis in f.axes for l in axis.get_lines()]
for l, true_value in zip(lines[::-2], rn.kinetic_parameters['true_value'].tolist()):
    y = l.get_ydata()[0] - 1.15
    p = ax.plot(true_value, y, '+', color='r')
ax.legend([p[0], lines[1], lines[0]], ['True value', 'Prior', 'Posterior'], frameon=False, loc='upper left')
ax.semilogx()
plt.show()

The next cell makes a similar plot for the observed metabolite concentrations. Here the convergence towards the true value from prior to posterior is much higher than for the marginal parameter distributions. 

In [ ]:
posterior_low, posterior_med, posterior_high = (
    infd_posterior.posterior_predictive['simulated_concentration_measurement'].quantile(q, dim=['chain', 'draw'])
    for q in [0.1, 0.5, 0.9]
)
prior_low, prior_med, prior_high = (
    infd_prior.posterior_predictive['simulated_concentration_measurement'].quantile(q, dim=['chain', 'draw'])
    for q in [0.1, 0.5, 0.9]
)
obs = infd_posterior.observed_data['simulated_concentration_measurement']
x = np.linspace(infd_posterior.posterior_predictive['simulated_concentration_measurement'].min(), 
                infd_posterior.posterior_predictive['simulated_concentration_measurement'].max(), 100)
labels = posterior_med.coords['concentration_measurements']

f, ax = plt.subplots(figsize=[15, 5])

ax.scatter(prior_med, obs, label='', color='tab:orange')
ax.hlines(obs, prior_low, prior_high, color='tab:orange', label='Prior')

ax.scatter(posterior_med, obs, label='', color='tab:blue')
ax.hlines(obs, posterior_low, posterior_high, color='tab:blue', label='Posterior', alpha=1)
for label, observed_value, prior_med_value in zip(labels.values, obs.values, prior_med.values):
    ax.text(prior_med_value, observed_value * 1.1, label)


ax.plot(x, x, color='r', linestyle='--', label='y=x')
ax.semilogx()
ax.semilogy()
legend = ax.legend(frameon=False)
text = ax.set(title='How did the metabolite concentration predictions change from prior to posterior?',
              xlabel='Modelled value',
              ylabel='Observed value')

Finally, the following plot shows the posterior correlation between each parameter. We can see that although there are some correlations, there are no troubling funnel or banana shapes.

In [ ]:
axes = arviz.plot_pair(infd_posterior, 
                       var_names=['kinetic_parameter'], 
                       coords={'parameter_names': ['r1.Kcat1', 'r1.Ka', 'r1.Kcat2', 'r2.Kcat1']},
                       figsize=[15, 10])


## Acknowledgements
The research was produced as part of the project '[Quantitative modelling of cell metabolism](https://www.biosustain.dtu.dk/research/scientific-sections/quantitative-modelling-of-cell-metabolism)', which is part of the DTU Biosustain department and is funded by the [Novo Nordisk Foundation](https://novonordiskfonden.dk/en/).


## References

Cleland, W. W. 1963. “The Kinetics of Enzyme-Catalyzed Reactions with Two or More Substrates or Products: I. Nomenclature and Rate Equations.” _Biochimica et Biophysica Acta (BBA) - Specialized Section on Enzymological Subjects 67_ (January): 104–37. doi:10.1016/0926-6569(63)90211-6.

Popova, S. V., and E. E. Sel’kov. 1975. “Generalization of the Model by Monod, Wyman and Changeux for the Case of a Reversible Monosubstrate Reaction.” _FEBS Letters_ 53 (3): 269–73. doi:10.1016/0014-5793(75)80034-2.

Saa, Pedro A., and Lars K. Nielsen. 2016. “Construction of Feasible and Accurate Kinetic Models of Metabolism: A Bayesian Approach.” _Scientific Reports_ 6 (1). doi:10.1038/srep29635.

———. 2017. “Formulation, Construction and Analysis of Kinetic Models of Metabolism: A Review of Modelling Frameworks.” _Biotechnology Advances_ 35 (8): 981–1003. doi:10.1016/j.biotechadv.2017.09.005.

Tsigkinopoulou, Areti, Syed Murtuza Baker, and Rainer Breitling. 2017. “Respectful Modeling: Addressing Uncertainty in Dynamic System Models for Molecular Biology.” _Trends in Biotechnology_ 35 (6): 518–29. doi:10.1016/j.tibtech.2016.12.008.
